# ANSWERING BUSINESS QUESTIONS WITH SQL

In [2]:
# Libraries
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine

In [3]:
# Connect to database
db = create_engine( 'sqlite:///db_olist_sqlite' )
conn = db.connect()

In [16]:
# Check database
query = '''
    SELECT 
        type, name, sql
    FROM sqlite_master
    WHERE type = 'table'
'''
table = pd.read_sql_query( query, conn )
table

,type,name,sql
0,table,customers,CREATE TABLE customers(\n customer_id ...
1,table,geolocation,CREATE TABLE geolocation(\n geolocation...
2,table,order_items,CREATE TABLE order_items(\n order_id ...
3,table,order_payments,CREATE TABLE order_payments(\n order_id...
4,table,order_reviews,CREATE TABLE order_reviews(\n review_id...
5,table,orders,CREATE TABLE orders(\n order_id ...
6,table,products,CREATE TABLE products(\n product_id ...
7,table,sellers,CREATE TABLE sellers(\n seller_id ...
8,table,product_category_name_translation,CREATE TABLE product_category_name_translation...


## 01. How many orders were placed for each payment type?

In [42]:
query = '''
    SELECT 
        payment_type,
        count( order_id ) AS num_orders
    FROM 
        order_payments
    GROUP BY 
        payment_type
    ORDER BY
        num_orders DESC
'''
pd.read_sql_query( query, conn )

,payment_type,num_orders
0,credit_card,153590
1,boleto,39568
2,voucher,11550
3,debit_card,3058
4,not_defined,6


## 02. What is the maximum and minimum number of installments in payments?

In [43]:
query = '''
    SELECT 
        max( payment_installments ) AS max_installments,
        min( payment_installments ) AS min_installments
    FROM 
        order_payments
'''
pd.read_sql_query( query, conn )

,max_installments,min_installments
0,24,0


## 03. What are the top 10 orders with the highest values?

In [25]:
query = '''
    SELECT 
        *
    FROM 
        order_payments
    ORDER BY
        payment_value DESC
    LIMIT 10
'''
pd.read_sql_query( query, conn )

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,03caa2c082116e1d31e67e9ae3700499,1,credit_card,1,13664.08
1,03caa2c082116e1d31e67e9ae3700499,1,credit_card,1,13664.08
2,736e1922ae60d0d6a89247b851902527,1,boleto,1,7274.88
3,736e1922ae60d0d6a89247b851902527,1,boleto,1,7274.88
4,0812eb902a67711a1cb742b3cdaa65ae,1,credit_card,8,6929.31
5,0812eb902a67711a1cb742b3cdaa65ae,1,credit_card,8,6929.31
6,fefacc66af859508bf1a7934eab1e97f,1,boleto,1,6922.21
7,fefacc66af859508bf1a7934eab1e97f,1,boleto,1,6922.21
8,f5136e38d1a14a4dbd87dff67da82701,1,boleto,1,6726.66
9,f5136e38d1a14a4dbd87dff67da82701,1,boleto,1,6726.66


## 04. What are the last 10 orders with the lowest values?

In [36]:
query = '''
    SELECT 
        *
    FROM 
        order_payments
    WHERE
       payment_value > 0
    ORDER BY
        payment_value ASC
    LIMIT 10
'''
pd.read_sql_query( query, conn )

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,7db5f2eb8f5f54db9f9e71ba4296bcbf,2,voucher,1,0.01
1,fb4de3600d359f84927517e78ff9ba54,1,voucher,1,0.01
2,0218c7a4fb8d5b1bd22c82b783b8359c,1,credit_card,1,0.01
3,ca4b9f3ce6fc19e8533501cf8c6b832e,1,credit_card,1,0.01
4,636f0241ddc83a3b9e37a8088167bd45,2,voucher,1,0.01
5,25b5b0ea53b7d5a2d5712a0d9d0b3649,1,credit_card,1,0.01
6,7db5f2eb8f5f54db9f9e71ba4296bcbf,2,voucher,1,0.01
7,fb4de3600d359f84927517e78ff9ba54,1,voucher,1,0.01
8,0218c7a4fb8d5b1bd22c82b783b8359c,1,credit_card,1,0.01
9,ca4b9f3ce6fc19e8533501cf8c6b832e,1,credit_card,1,0.01


## 05. What is the average payment amount per payment type?

In [41]:
query = '''
    SELECT 
        payment_type, 
        avg( payment_value ) AS average_payments
    FROM 
        order_payments
    GROUP BY
        payment_type
    ORDER BY
        average_payments DESC
'''
pd.read_sql_query( query, conn )

,payment_type,average_payments
0,credit_card,163.319021
1,boleto,145.034435
2,debit_card,142.570170
3,voucher,65.703354
4,not_defined,0.000000


## 06. What are the top 5 customers with the highest payment amounts on boleto?

In [129]:
query = '''
    SELECT 
        distinct( c.customer_unique_id ),
        op.payment_type, 
        op.payment_value
    FROM 
        order_payments op LEFT JOIN orders o    ON ( o.order_id = op.order_id )
                          LEFT JOIN customers c ON ( c.customer_id = o.customer_id )
    WHERE
        op.payment_type = 'boleto'
    ORDER BY
        op.payment_value DESC
    LIMIT 5
'''
pd.read_sql_query( query, conn )

,customer_unique_id,payment_type,payment_value
0,763c8b1c9c68a0229c42c9fc6f662b93,boleto,7274.88
1,459bef486812aa25204be022145caa62,boleto,6922.21
2,ff4159b92c40ebe40454e3e6a7c35ed6,boleto,6726.66
3,4007669dec559734d6f53e029e360987,boleto,6081.54
4,f0767ae738c3d90e7b737d7b8b8bb4d1,boleto,3979.55


## 07. What are the top 5 customers with the highest credit card payment amounts?

In [128]:
query = '''
    SELECT 
        distinct( c.customer_unique_id ),
        op.payment_type, 
        op.payment_value
    FROM 
        order_payments op LEFT JOIN orders o    ON ( o.order_id = op.order_id )
                          LEFT JOIN customers c ON ( c.customer_id = o.customer_id )
    WHERE
        op.payment_type = 'credit_card'
    ORDER BY
        op.payment_value DESC
    LIMIT 5
'''
pd.read_sql_query( query, conn )

,customer_unique_id,payment_type,payment_value
0,0a0a92112bd4c708ca5fde585afaa872,credit_card,13664.08
1,dc4802a71eae9be1dd28f5d788ceb526,credit_card,6929.31
2,da122df9eeddfedc1dc1f5349a1a690c,credit_card,4950.34
3,5d0a2980b292d049061542014e8960bf,credit_card,4809.44
4,eebb5dda148d3893cdaf5b5ca3040ccb,credit_card,4764.34


## 08. What are the 10 most expensive products?

In [70]:
query = '''
    SELECT
        distinct ( product_id ),
        price
    FROM
        order_items
    ORDER BY
        price DESC
    LIMIT 10
'''
pd.read_sql_query( query, conn )

,product_id,price
0,489ae2aa008f021502940f251d4cce7f,6735.00
1,69c590f7ffc7bf8db97190b6cb6ed62e,6729.00
2,1bdf5e6731585cf01aa8169c7028d6ad,6499.00
3,a6492cc69376c469ab6f61d8f44de961,4799.00
4,c3ed642d592594bb648ff4a04cee2747,4690.00
5,259037a6a41845e455183f89c5035f18,4590.00
6,a1beef8f3992dbd4cd8726796aa69c53,4399.87
7,6cdf8fc1d741c76586d8b6b15e9eef30,4099.99
8,dd113cb02b2af9c8e5787e8f1f0722f6,4059.00
9,6902c1962dd19d540807d0ab8fade5c6,3999.90


## 09. What are the 10 cheapest products?

In [71]:
query = '''
    SELECT
        distinct ( product_id ),
        price
    FROM
        order_items
    ORDER BY
        price ASC
    LIMIT 10
'''
pd.read_sql_query( query, conn )

,product_id,price
0,8a3254bee785a526d548a81a9bc3c9be,0.85
1,270516a3f41dc035aa87d220228f844c,1.20
2,05b515fdc76e888aada3c6d66c201dff,1.20
3,46fce52cef5caa7cc225a5531c946c8b,2.20
4,310dc32058903b6416c71faff132df9e,2.29
5,680cc8535be7cc69544238c1d6a83fe8,2.90
6,2e8316b31db34314f393806fd7b6e185,2.99
7,44d53f1240d6332232e4393c06500475,3.00
8,29781581fb82fe2389560a3a5331d0ee,3.06
9,1716ea399ed8ee62ba811e6f55180f45,3.49


## 10. What are the 10 most purchased categories?

In [79]:
query = '''
    SELECT 
        p.product_category_name,
        count( distinct( oi.order_id ) ) AS num_orders
    FROM 
        order_items oi LEFT JOIN products p ON ( oi.product_id = p.product_id )
    GROUP BY
        p.product_category_name
    ORDER BY
        num_orders DESC
    LIMIT 10
'''
pd.read_sql_query ( query, conn )

,product_category_name,num_orders
0,cama_mesa_banho,9417
1,beleza_saude,8836
2,esporte_lazer,7720
3,informatica_acessorios,6689
4,moveis_decoracao,6449
5,utilidades_domesticas,5884
6,relogios_presentes,5624
7,telefonia,4199
8,automotivo,3897
9,brinquedos,3886


## 11. What are the 5 products with the most reviews?

In [101]:
query = '''
    SELECT
        oi.product_id,
        count( orv.review_id ) AS num_reviews   
    FROM 
        order_items oi LEFT JOIN order_reviews orv ON ( oi.order_id = orv.order_id )
    WHERE
        orv.review_comment_message != 'null'
    GROUP BY
        oi.product_id
    ORDER BY
        num_reviews DESC
    LIMIT 5
'''
pd.read_sql_query( query, conn )

,product_id,num_reviews
0,99a4788cb24856965c36a24e339b6058,948
1,422879e10f46682990de24d770e7f83d,916
2,368c6c730842d78016ad823897a372db,836
3,53759a2ecddad2bb87a079a1f1519f73,784
4,aca2eb7d00ea1a7b8ebd4e68314663af,776


## 12. What are the top 10 products without any reviews?

In [178]:
query = '''
    SELECT
        oi.product_id,
        count( orv.review_id ) AS num_null_reviews   
    FROM 
        order_items oi LEFT JOIN order_reviews orv ON ( oi.order_id = orv.order_id )
    WHERE
        orv.review_comment_message is null
    GROUP BY
        oi.product_id
    ORDER BY
        num_null_reviews DESC
    LIMIT 10
'''
pd.read_sql_query( query, conn )

,product_id,num_null_reviews
0,aca2eb7d00ea1a7b8ebd4e68314663af,1320
1,422879e10f46682990de24d770e7f83d,1028
2,99a4788cb24856965c36a24e339b6058,980
3,53b36df67ebb7c41585e8d54d6772e08,824
4,389d119b48cf3043d311335e499d9c6b,788
5,d1c427060a0f73f6b889a5c7c61f2ac4,780
6,368c6c730842d78016ad823897a372db,716
7,53759a2ecddad2bb87a079a1f1519f73,708
8,154e7e31ebfa092203795c972e5804a6,700
9,3dd2a17168ec895c781a9191c1e95ad7,648


## 13. What are the 10 customers with the highest number of orders?

In [126]:
query = '''
    SELECT
        customer_unique_id,
        count ( distinct( order_id ) ) AS num_orders
    FROM 
        orders o LEFT JOIN customers c ON ( o.customer_id = c.customer_id )
    GROUP BY
        customer_unique_id
    ORDER BY
        num_orders DESC
    LIMIT 10
'''
pd.read_sql_query( query, conn )

,customer_unique_id,num_orders
0,8d50f5eadf50201ccdcedfb9e2ac8455,17
1,3e43e6105506432c953e165fb2acf44c,9
2,ca77025e7201e3b30c44b472ff346268,7
3,6469f99c1f9dfae7733b25662e7f1782,7
4,1b6c7548a2a1f9037c1fd3ddfed95f33,7
5,f0e310a6839dce9de1638e0fe5ab282a,6
6,de34b16117594161a6a89c50b289d35a,6
7,dc813062e0fc23409cd255f7f53c7074,6
8,63cfc61cee11cbe306bff5857d00bfe4,6
9,47c1a3033b8b77b3ab6e109eb4d5fdf3,6


## 14. Which 10 customers have the least amount of orders?

In [130]:
query = '''
    SELECT
        customer_unique_id,
        count ( distinct( order_id ) ) AS num_orders
    FROM 
        orders o LEFT JOIN customers c ON ( o.customer_id = c.customer_id )
    GROUP BY
        customer_unique_id
    ORDER BY
        num_orders ASC
    LIMIT 10
'''
pd.read_sql_query( query, conn )

,customer_unique_id,num_orders
0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1
2,0000f46a3911fa3c0805444483337064,1
3,0000f6ccb0745a6a4b88665a16c9f078,1
4,0004aac84e0df4da2b147fca70cf8255,1
5,0004bd2a26a76fe21f786e4fbd80607f,1
6,00050ab1314c0e55a6ca13cf7181fecf,1
7,00053a61a98854899e70ed204dd4bafe,1
8,0005e1862207bf6ccc02e4228effd9a0,1
9,0005ef4cd20d2893f0d9fbd94d3c0d97,1


## 15. What vendors are there on the base?

In [133]:
query = '''
    SELECT
        distinct( seller_id )
    FROM 
        sellers
'''
pd.read_sql_query( query, conn )

,seller_id
0,3442f8959a84dea7ee197c632cb2df15
1,d1b65fc7debc3361ea86b5f14c68d2e2
2,ce3ad9de960102d0677a81f5d0bb7b2d
3,c0f3eea2e14555b6faeea3dd58c1b1c3
4,51a04a8a6bdcb23deccc82b0b80742cf
...,...
3090,98dddbc4601dd4443ca174359b237166
3091,f8201cab383e484733266d1906e2fdfa
3092,74871d19219c7d518d0090283e03c137
3093,e603cf3fec55f8697c9059638d6c8eb5


## 16. What is the distribution of sellers by state?

In [136]:
query = '''
    SELECT
        seller_state,
        count( distinct( seller_id )) AS num_seller
    FROM 
        sellers
    GROUP BY
        seller_state
    ORDER BY
        num_seller DESC
'''
pd.read_sql_query( query, conn )

,seller_state,num_seller
0,SP,1849
1,PR,349
2,MG,244
3,SC,190
4,RJ,171
5,RS,129
6,GO,40
7,DF,30
8,ES,23
9,BA,19


## 17. What is the distribution of customers by state?

In [138]:
query = '''
    SELECT
       customer_state,
       count( distinct( customer_unique_id ) ) AS num_customers
    FROM customers
    GROUP BY
        customer_state
    ORDER BY
        num_customers DESC        
'''
pd.read_sql_query( query, conn )

,customer_state,num_customers
0,SP,40302
1,RJ,12384
2,MG,11259
3,RS,5277
4,PR,4882
5,SC,3534
6,BA,3277
7,DF,2075
8,ES,1964
9,GO,1952


## 18. What are the top 10 sellers that received the most payments by boleto?

In [151]:
query = '''
    SELECT
       seller_id,
       payment_type,
       SUM( payment_value ) AS total_payments       
    FROM
        order_items oi LEFT JOIN order_payments op ON ( op.order_id = oi.order_id )
    WHERE
        payment_type = 'boleto'
    GROUP BY
        seller_id
    ORDER BY
        payment_value DESC
    LIMIT 10        
'''
pd.read_sql_query( query, conn )

,seller_id,payment_type,total_payments
0,80ceebb4ee9b31afb6c6a916a574a1e2,boleto,27688.84
1,ee27a8f15b1dded4d213a468ba4eb391,boleto,26906.64
2,039e6ad9dae79614493083e241147386,boleto,15918.20
3,e2a1ac9bf33e5549a2a4f834e70df2f8,boleto,24263.24
4,d63c73efd41eb002280e7ec831424edb,boleto,11752.68
5,abe021b01ba992245271b9aa422032df,boleto,11178.00
6,d9e8c084b68fe958861d8f2c21202e6b,boleto,23214.08
7,59417c56835dd8e2e72f91f809cd4092,boleto,30368.80
8,6fa9202c10491e472dffd59a3e82b2a3,boleto,21706.88
9,2528744c5ef5d955adc318720a94d2e7,boleto,9263.32


## 19. What are the 10 worst sellers in terms of number of sales?

In [157]:
query = '''
    SELECT
       oi.seller_id,
       count( distinct( o.order_id ) ) AS num_orders
    FROM
        orders o LEFT JOIN order_items oi ON ( o.order_id = oi.order_id )
    GROUP BY
        seller_id
    ORDER BY
        num_orders ASC
    LIMIT 10        
'''
pd.read_sql_query( query, conn )

,seller_id,num_orders
0,001e6ad469a905060d959994f1b41e4f,1
1,003554e2dce176b5555353e4f3555ac8,1
2,00ab3eff1b5192e5f1a63bcecfee11c8,1
3,00d8b143d12632bad99c0ad66ad52825,1
4,010da0602d7774602cd1b3f5fb7b709e,1
5,011b0eaba87386a2ae96a7d32bb531d1,1
6,028872bfa080090a9d0abd4f1af168f8,1
7,0336182e1b3e92f029d5354832045fdf,1
8,04843805947f0fc584fc1969b6e50fe7,1
9,04ee0ec01589969663ba5967c0e0bdc0,1


## 20. How many products are purchased on average per order?

In [171]:
query = '''
    WITH orders_and_products AS (
        SELECT
           o.order_id,
           count( oi.product_id ) AS num_products
        FROM
            orders o LEFT JOIN order_items oi ON ( o.order_id = oi.order_id )
        GROUP BY
            o.order_id   
        ORDER BY
            num_products
    )
    SELECT
        avg( num_products ) AS average_products_per_order
    FROM
        orders_and_products
'''
pd.read_sql_query( query, conn )

,average_products_per_order
0,4.53133


In [179]:
# Closing connection
conn.close()